# 1.0 Extract movement and track positions over time.

For each video we use YOLOv8 to extract movement data as a set of body keypoints and use its `model.track` method to track individuals over time.


# 1.1 Video pose estimation with YOLOv8

[YOLOv8](https://github.com/ultralytics/ultralytics) is a commercially maintained version of the YOLO object recognition model. [Yolov7](https://github.com/WongKinYiu/yolov7) introduced pose estimation and v8 improves the models and makes everything much more user-friendly. It can be installed as a package

* Pip : `pip install ultralytics`
* Conda : `conda install -c conda-forge ultralytics`

## 1.2 Object tracking 

YoloV8 also comes with a `model.track` method. This aims to keep track of all identified objects over the course of a video. Let's make use of that to track individuals over time. 

This is pretty easy instead of calling 
`results = model(video_path, stream=True)`

we can call
`results = model.track(video_path, stream=True)`

https://docs.ultralytics.com/modes/track/#persisting-tracks-loop

In [ ]:
import os
import math
import sys
import time
import pandas as pd
import numpy as np
import cv2
import torch
from ultralytics import YOLO

import utils

In [ ]:
videos_in = r"..\LookitLaughter.test"
metadata_file = "_LookitLaughter.xlsx"
data_dir = r"..\data\1_interim"

#get metadata from excel file
metadata = pd.read_excel(os.path.join(videos_in, metadata_file))
metadata.head()

In [ ]:
#get yolo model with pose estimation
model = YOLO('yolov8n-pose.pt')

In [ ]:
processedvideos = utils.getprocessedvideos(data_dir)
processedvideos.head()

In [ ]:
#loop through each row of metadata and
#process all related videos
forcemetadata = False
forceprocess = False
tracking = True

for index, mrow in metadata.iterrows():
    #get VIDEOID from first column of metadata
    videoname = mrow["VideoID"]
    stemname = os.path.splitext(videoname)[0]
    print(f"video:{videoname}")

    #check we want to refill metadata or this video is not already in processedvideos dataframe
    if forcemetadata or videoname not in processedvideos["VideoID"].values: 
        #use cv2 to get fps and other video info to add to dataframe
        cap = cv2.VideoCapture(os.path.join(videos_in,videoname))    
        if (cap.isOpened()== False):
            print("Error opening video stream or file")
            continue
        else:
            #add row to processedvideos dataframe
            row = {"VideoID":videoname,
                "ChildID":mrow["ChildID"],
                "JokeType":mrow["JokeType"],
                "JokeNum":mrow["JokeNum"],
                "JokeRep":mrow["JokeRep"],
                "JokeTake":mrow["JokeTake"],
                "HowFunny":mrow["HowFunny"],
                "LaughYesNo":mrow["LaughYesNo"],
                "Frames":cap.get(cv2.CAP_PROP_FRAME_COUNT),
                "FPS":cap.get(cv2.CAP_PROP_FPS) , 
                "Width":cap.get(cv2.CAP_PROP_FRAME_WIDTH), 
                "Height":cap.get(cv2.CAP_PROP_FRAME_HEIGHT), 
                "Duration":cap.get(cv2.CAP_PROP_FRAME_COUNT)/cap.get(cv2.CAP_PROP_FPS)
                }
            cap.release()
            print(f"Adding video info: {row}")
            newrow = pd.DataFrame(row, index=[0])
            processedvideos = pd.concat([processedvideos,newrow], ignore_index=True)

    #select the dataframe row for this video 
    row = processedvideos.loc[processedvideos["VideoID"] == videoname]
    #is this video in the processedvideos dataframe?
    if row.empty:
        print(f"error: processsedvideos.xlsx has no row for {videoname}")
        continue
    #has this video already been processed and can we find the csv file?
    if not forceprocess and not pd.isnull(row["Keypoints.file"].values[0]) and os.path.exists(row["Keypoints.file"].values[0]):
        print(f"already processed {videoname} results in {row['Keypoints.file'].values[0]}")
        continue
    else:
        #use ultralytics YOLO to get keypoints
        keypointsdf =utils.videotokeypoints(model, os.path.join(videos_in,videoname) , track = True)
        #save keypointsdf as csv    
        keypointspath = data_dir + "\\" + stemname + ".csv"
        keypointsdf.to_csv(keypointspath)
        row["Keypoints.file"] = keypointspath
        row["Keypoints.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[processedvideos["VideoID"] == videoname] = row
    

    #update processedvideos excel file
    utils.saveprocessedvidoes(processedvideos, data_dir)

In [9]:
processedvideos.head()

#Todo - something wrong with indexing of processedvideos dataframe, extra unnamed column appearing. 

,Unnamed: 0,Index,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,...,Audio.file,Faces.when,Faces.file,LastError,Speech.file,Speech.when,Objects.file,Objects.when,Understand.file,Understand.when
0,0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,...,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-24 07:52:41,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38,NaN,NaN,NaN,NaN
1,1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,...,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-24 07:54:38,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39,NaN,NaN,NaN,NaN
2,2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,...,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-24 07:55:58,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
3,3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,...,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-24 07:56:57,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
4,4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,...,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-24 07:59:00,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48,NaN,NaN,NaN,NaN
